In [6]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [17]:
data=pd.read_csv('data/spam.csv', encoding='latin1')
data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [18]:
data.shape

(5572, 2)

In [19]:
#check for duplicates and remove them
data.drop_duplicates(inplace = True)

In [20]:
data.shape

(5169, 2)

In [21]:
#show the number of missing (NAN, NaN, nan) data for each column
data.isnull().sum()

v1    0
v2    0
dtype: int64

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahta\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [32]:
def process_text(v2):
    #remove punctuations
    #remove stopwords
    #return a list of clean text words
    nopunc = [char for char in v2 if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    return clean_words

In [33]:
data['v2'].head().apply(process_text)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: v2, dtype: object

In [34]:
#conevert a collection of text to  matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
messages_bow = CountVectorizer(analyzer = process_text).fit_transform(data['v2'])

In [38]:
#split the data into 80 % training and 20 % testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(messages_bow, data['v1'], test_size=0.2, random_state=0)

In [39]:
messages_bow.shape

(5169, 11304)

In [41]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train, Y_train)

In [43]:
print(classifier.predict(X_train))
print(Y_train.values)

['ham' 'ham' 'ham' ... 'ham' 'ham' 'ham']
['ham' 'ham' 'ham' ... 'ham' 'ham' 'ham']


In [51]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_train)
print(classification_report(Y_train, pred))
print('Counfusion Matrix: \n', confusion_matrix(Y_train, pred))
print('Acccuracy: \n', accuracy_score(Y_train, pred))

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00      3631
        spam       0.98      0.98      0.98       504

    accuracy                           1.00      4135
   macro avg       0.99      0.99      0.99      4135
weighted avg       1.00      1.00      1.00      4135

Counfusion Matrix: 
 [[3623    8]
 [  11  493]]
Acccuracy: 
 0.9954050785973397


In [52]:
print(classifier.predict(X_test))
print(Y_test.values)

['ham' 'ham' 'ham' ... 'ham' 'ham' 'ham']
['ham' 'ham' 'ham' ... 'ham' 'ham' 'ham']


In [53]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(Y_test, pred))
print('Counfusion Matrix: \n', confusion_matrix(Y_test, pred))
print('Acccuracy: \n', accuracy_score(Y_test, pred))

              precision    recall  f1-score   support

         ham       0.99      0.96      0.97       885
        spam       0.80      0.93      0.86       149

    accuracy                           0.96      1034
   macro avg       0.89      0.94      0.92      1034
weighted avg       0.96      0.96      0.96      1034

Counfusion Matrix: 
 [[850  35]
 [ 11 138]]
Acccuracy: 
 0.9555125725338491
